In [ ]:
"""
Created on Tue Aug 18 22:21:40 2020

@author: FisaGol
"""


In [ ]:
!pip install numerapi
!pip install yfinance
!pip install simplejson

     |████████████████████████████████| 133kB 4.2MB/s 


In [ ]:

import numerapi
import yfinance
import simplejson

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests as re
from datetime import datetime
from dateutil.relativedelta import relativedelta, FR
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.stattools import adfuller
import pandas as pd
import scipy
import numpy as np
import numerapi
import random
import sklearn
import xgboost
from sklearn import (
    feature_extraction, feature_selection, decomposition, linear_model,
    model_selection, metrics, svm, preprocessing, utils
)
import numpy as np
import matplotlib.pyplot as plt
from numpy import array
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,MinMaxScaler, OrdinalEncoder, LabelEncoder,OneHotEncoder
import warnings
from pandas import read_csv
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential, model_from_json, load_model
from keras.layers import Dense, Dropout, Activation,LSTM,Bidirectional, MaxPooling2D, Flatten,GRU
from keras.optimizers import SGD
from keras.regularizers import l2
from sklearn.model_selection import StratifiedKFold, GroupKFold, GridSearchCV,cross_val_score, RepeatedStratifiedKFold,train_test_split
from sklearn.metrics import log_loss, make_scorer, mean_squared_error,classification_report,accuracy_score
from keras.layers.normalization import BatchNormalization
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
import tensorflow as tf
from scipy import ndimage 
from keras.utils import np_utils 
from sklearn.model_selection import KFold 
import keras
from keras.optimizers import SGD,Adam
from sklearn.ensemble import GradientBoostingRegressor



/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
seed = 7
rand = np.random.seed(seed)

#https://colab.research.google.com/drive/1ECh69C0LDCUnuyvEmNFZ51l_276nkQqo#scrollTo=Ea1f_GxAbOqC

#https://github.com/leonhma/yfinance

def RSI(prices, interval=10):
    '''Computes Relative Strength Index given a price series and lookback interval
  Modified from https://stackoverflow.com/questions/20526414/relative-strength-index-in-python-pandas
  See more here https://www.investopedia.com/terms/r/rsi.asp'''
    delta = prices.diff()

    dUp, dDown = delta.copy(), delta.copy()
    dUp[dUp < 0] = 0
    dDown[dDown > 0] = 0

    RolUp = dUp.rolling(interval).mean()
    RolDown = dDown.rolling(interval).mean().abs()

    RS = RolUp / RolDown
    RSI = 100.0 - (100.0 / (1.0 + RS))
    return RSI


In [ ]:
##############################################################################
########################## DOWLOAD DATA ######################################
##############################################################################

In [ ]:
# read in yahoo to bloomberg ticker map, still a work in progress, h/t wsouza
ticker_map = pd.read_csv('https://numerai-signals-public-data.s3-us-west-2.amazonaws.com/signals_ticker_map_w_bbg.csv')

In [ ]:
ticker_map

,ticker,bloomberg_ticker,yahoo
0,000060 KS,000060 KS,000060.KS
1,000080 KS,000080 KS,000080.KS
2,000100 KS,000100 KS,000100.KS
3,000120 KS,000120 KS,000120.KS
4,000150 KS,000150 KS,000150.KS
...,...,...,...
5026,ZUMZ,ZUMZ US,ZUMZ
5027,ZUO,ZUO US,ZUO
5028,ZURN SW,ZURN SW,ZURN.SW
5029,ZYXI,ZYXI US,ZYXI


In [ ]:
numerai_tickers = ticker_map.bloomberg_ticker
yfinance_tickers = ticker_map.yahoo

print(f"Number of eligible tickers in map: {len(ticker_map)}")

Number of eligible tickers in map: 5031


In [ ]:
n = 300  #chunk row size
chunk_df = [yfinance_tickers.iloc[i:i+n] for i in range(0, len(yfinance_tickers), n)]

In [ ]:
chunk_df

In [ ]:
concat_dfs = []
for df in chunk_df:
    try:
        # set threads = True for faster performance, but tickers will fail, scipt may hang
        # set threads = False for slower performance, but more tickers will succeed
        temp_df = yfinance.download(df.str.cat(sep=' '), start='2002-01-01', threads=False)
        temp_df = temp_df['Adj Close'].stack().reset_index()
        concat_dfs.append(temp_df)
    except simplejson.errors.JSONDecodeError:
        pass

full_data = pd.concat(concat_dfs)

In [ ]:
##############################################################################
########################## DATA WRANGLING ####################################
##############################################################################


# properly position and clean raw data, after taking adjusted close only
full_data.columns = ['date', 'ticker', 'price']
full_data.set_index('date', inplace=True)

# convert yahoo finance tickers back to numerai tickers
full_data['bloomberg_ticker'] = full_data.ticker.map(dict(zip(yfinance_tickers, numerai_tickers)))

print(f"Number of tickers with data: {len(full_data.ticker.unique())}")
full_data.head()



In [ ]:
##############################################################################
########################## ENGINEER FEATURES #################################
##############################################################################



ticker_groups = full_data.groupby('ticker')
full_data['RSI'] = ticker_groups['price'].transform(lambda x: RSI(x))

# group by era (date) and create quintile labels within each era, useful for learning relative ranking
date_groups = full_data.groupby(full_data.index)
full_data['RSI_quintile'] = date_groups['RSI'].transform(lambda group: pd.qcut(group, 5, labels=False, duplicates='drop'))
full_data.dropna(inplace=True)

full_data.head()


ticker_groups = full_data.groupby('ticker')


#create lagged features, lag 0 is that day's value, lag 1 is yesterday's value, etc
num_days = 5
for day in range(num_days+1):
    full_data[f'RSI_quintile_lag_{day}'] = ticker_groups['RSI_quintile'].transform(lambda group: group.shift(day))

full_data.tail()



# create difference of the lagged features (change in RSI quintile by day)
for day in range(num_days):
    full_data[f'RSI_diff_{day}'] = full_data[f'RSI_quintile_lag_{day}'] - full_data[f'RSI_quintile_lag_{day + 1}']
    full_data[f'RSI_abs_diff_{day}'] = np.abs(full_data[f'RSI_quintile_lag_{day}'] - full_data[f'RSI_quintile_lag_{day + 1}'])

full_data.tail()

# define column names of features, target, and prediction
feature_names = [f'RSI_quintile_lag_{num}' for num in range(num_days)] + [f'RSI_diff_{num}' for num in range(num_days)] + [f'RSI_abs_diff_{num}' for num in range(num_days)]

print(f'Features for training:\n {feature_names}')


In [ ]:
##############################################################################
########################## NUMERAI TARGETS ###################################
##############################################################################


# read in Signals targets
numerai_targets = 'https://numerai-signals-public-data.s3-us-west-2.amazonaws.com/signals_train_val.csv'
targets = pd.read_csv(numerai_targets)
targets['date'] = pd.to_datetime(targets['friday_date'], format='%Y%m%d')

targets.columns = ['bloomberg_ticker', 'friday_date', 'data_type','target','date']

targets.head()

# the number of tickers per era has generally increased
targets.groupby('date').apply(lambda x: len(x)).plot(kind='line', figsize=(10,4), title='Number of tickers per era')

# the target classes are imbalanced, but we can treat this like a regression problem
targets.target.value_counts()

# the imbalance is consistent across eras with a constant class ratio of: 5%, 20%, 50%, 20%, 5%
pivot_target = targets.groupby(['date','target']).apply(lambda x: len(x)).reset_index(1).pivot(columns='target',values=0)
pivot_target.iloc[::20].plot(kind='bar', stacked=True, figsize=(9,3), title='Number of tickers in each class per era')

stacked_data = pivot_target.apply(lambda x: x/sum(x), axis=1)
stacked_data.iloc[::20].plot(kind='bar', stacked=True, figsize=(9,3), title='Proportion of tickers in each class per era')



In [ ]:
##############################################################################
########################## COMBINE FEATURES AND TARGETS ######################
##############################################################################


ML_data = pd.merge(full_data.reset_index(), targets, on=['date', 'bloomberg_ticker']).set_index('date')


# for training and testing we want clean, complete data only
ML_data.dropna(inplace=True)
ML_data = ML_data[ML_data.index.weekday==4] # ensure we have only fridays
ML_data = ML_data[ML_data.index.value_counts() > 200] # drop eras with under 200 observations per era

print(f'Number of eras in data: {len(ML_data.index.unique())}')

ML_data.head()


In [ ]:
##############################################################################
#######################    TRAIN MODEL    ####################################
##############################################################################


#https://colab.research.google.com/drive/1ECh69C0LDCUnuyvEmNFZ51l_276nkQqo#scrollTo=8v1YekOYIMqO
# train test split
train_data = ML_data[ML_data['data_type'] == 'train']
test_data = ML_data[ML_data['data_type'] == 'validation']


# train model
print("Training model...")
model = GradientBoostingRegressor()
model.fit(train_data[feature_names], train_data['target'])
print("Model trained.")

plt.figure(figsize=(15,3))
plt.bar(feature_names, model.feature_importances_)
plt.xticks(rotation=70)
plt.show()


In [ ]:
##############################################################################
################### FORECAST TOURNAMENT DATA #################################
##############################################################################


train_data['prediction'] = model.predict(train_data[feature_names])
test_data['prediction'] = model.predict(test_data[feature_names])

#show prediction distribution, most should around the center
test_data['prediction'].hist(bins=30)



In [ ]:
##############################################################################
############################ EVALUATION ######################################
##############################################################################


TARGET_NAME = 'target'
PREDICTION_NAME = 'prediction'

def score(df):
    '''Takes df and calculates spearm correlation from pre-defined cols'''
    # method="first" breaks ties based on order in array
    return np.corrcoef(
        df[TARGET_NAME],
        df[PREDICTION_NAME].rank(pct=True, method="first")
    )[0,1]

def run_analytics(era_scores):
    print(f"Mean Correlation: {era_scores.mean():.4f}")
    print(f"Median Correlation: {era_scores.median():.4f}")
    print(f"Standard Deviation: {era_scores.std():.4f}")
    print('\n')
    print(f"Mean Pseudo-Sharpe: {era_scores.mean()/era_scores.std():.4f}")
    print(f"Median Pseudo-Sharpe: {era_scores.median()/era_scores.std():.4f}")
    print('\n')
    print(f'Hit Rate (% positive eras): {era_scores.apply(lambda x: np.sign(x)).value_counts()[1]/len(era_scores):.2%}')

    era_scores.rolling(10).mean().plot(kind='line', title='Rolling Per Era Correlation Mean', figsize=(15,4))
    plt.axhline(y=0.0, color="r", linestyle="--"); plt.show()

    era_scores.cumsum().plot(title='Cumulative Sum of Era Scores', figsize=(15,4))
    plt.axhline(y=0.0, color="r", linestyle="--"); plt.show()


# spearman scores by era
train_era_scores = train_data.groupby(train_data.index).apply(score)
test_era_scores = test_data.groupby(test_data.index).apply(score)

#train scores, in-sample and will be significantly overfit
run_analytics(train_era_scores)

#test scores, out of sample
run_analytics(test_era_scores)


In [ ]:
##############################################################################
###################  PREDICT AND SUBMIT ON LIVE DATA  ########################
##############################################################################


# choose data as of most recent friday
last_friday = datetime.now() + relativedelta(weekday=FR(-1))
date_string = last_friday.strftime('%Y-%m-%d')

live_data = full_data.loc[date_string].copy()
live_data.dropna(subset=feature_names, inplace=True)

print(f"Number of live tickers to submit: {len(live_data)}")

live_data.tail()

live_data[PREDICTION_NAME] = model.predict(live_data[feature_names])
live_data[PREDICTION_NAME].hist()

diagnostic_df = pd.concat([test_data, live_data])
diagnostic_df.tail()

diagnostic_df['friday_date'] = diagnostic_df.friday_date.fillna(last_friday.strftime('%Y%m%d')).astype(int)
diagnostic_df['data_type'] = diagnostic_df.data_type.fillna('live')
diagnostic_df[['bloomberg_ticker','friday_date','data_type','prediction']].reset_index(drop=True).to_csv('example_signal_upload.csv', index=False)


predictions = diagnostic_df[['bloomberg_ticker','friday_date','data_type', PREDICTION_NAME]].copy()
predictions.columns = ['bloomberg_ticker', 'friday_date', 'data_type','signal']

predictions.head()
predictions.tail()


In [ ]:
##############################################################################
######################### MAKE SUBMISSION #################################### 
##############################################################################


# Get your API keys and model_id from https://numer.ai/submit
public_id = "7TISUDGAWEVO2B35ECOQQXU2RWXGZN3I"
secret_key = "QJYUWIMFEEDNZ4GHUO6VSSKPMRCBFJIMJ7BZ65ESIWRN4YHGYHSRJDNL64TAG7EH"
model_id = "d49c26a4-aa5b-4490-9d58-300c5e05d996"
napi_signals = numerapi.SignalsAPI(public_id=public_id, secret_key=secret_key)
# Upload your predictions
predictions.to_csv("predictions.csv", index=False)
submission_id = napi_signals.upload_predictions("predictions.csv", model_id=model_id)